In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from igraph import *

# Import and start a Spark session.
from pyspark.sql import *
from pyspark.sql.functions import *
from nltk.tokenize.casual import TweetTokenizer
import string
from collections import Counter
from nltk.util import everygrams
spark = SparkSession.builder.getOrCreate()

In [2]:
from sklearn.externals import joblib
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import itertools as it

[nltk_data] Downloading package stopwords to /home/egj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
class Tweet:
    def __init__(_id,user,tweet):
        
        self.id = _id
        self.username = user
        self.tweet = tweet
    
    def extract_hashtags(tweet):
        
        pos = [pos for pos,char in enumerate(tweet) if char =='#']
        pos1 = [string.find(' ',p) for p in pos]
        indices = list(zip(pos,pos1))
        hashtags = [tweet[p[0]+1:p[1]] for p in indices]
        return hashtags

In [4]:
class User:
    def __init(_id,user):
        self.id = _id
        self.username = user

In [5]:
def extract_username_from_link(link):
    return link.split('/')[3]

In [6]:
DATA_PATH = 'tweets-notmypresident.json'
data = spark.read.json(DATA_PATH)

In [7]:
print(data.columns)
pd_data = data.toPandas()

['conversation_id', 'created_at', 'date', 'gif_thumb', 'gif_url', 'has_parent_tweet', 'hashtags', 'id', 'is_quote_status', 'is_reply_to', 'likes_count', 'link', 'location', 'mentions', 'name', 'photos', 'place', 'quote_id', 'quote_url', 'replies', 'replies_count', 'retweet', 'retweets_count', 'tags', 'time', 'timezone', 'tweet', 'urls', 'user_id', 'username', 'video_thumb', 'video_url']


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/egj/anaconda3/envs/ada/lib/python3.7/site-packages/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/egj/anaconda3/envs/ada/lib/python3.7/site-packages/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/home/egj/anaconda3/envs/ada/lib/python3.7/site-packages/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


In [8]:
pd_data.head(5)

,conversation_id,created_at,date,gif_thumb,gif_url,has_parent_tweet,hashtags,id,is_quote_status,is_reply_to,...,retweets_count,tags,time,timezone,tweet,urls,user_id,username,video_thumb,video_url
0,804112748397424640,1480550393000,2016-12-01,,,0,"[#notmypresident, #dumptrump]",804112748397424640,1,0,...,0,[],00:59:53,CET,Unfortunately @realDonaldTrump doesn't underst...,[https://twitter.com/washingtonpost/status/804...,25591312,i_create_things,,
1,804112672203669504,1480550375000,2016-12-01,,,0,"[#notmypresident, #stillwithher]",804112672203669504,1,0,...,0,[],00:59:35,CET,.@maddow U R the only one that can keep me awa...,[https://twitter.com/frankthorp/status/8040876...,27801842,jenk264,,
2,804061970819403776,1480550367000,2016-12-01,,,1,"[#twousefulidiots, #notmypresident]",804112641002192897,0,1,...,0,[],00:59:27,CET,@TheLastWord As if the VA doesn't have enough ...,[],522508993,rebelwriter57,,
3,804110522027126784,1480550358000,2016-12-01,,,1,[#notmypresident],804112600380407809,0,1,...,31,[],00:59:18,CET,God help us we've become an oligarchy. #NotMyP...,[],21827563,kbates1948,,
4,804112446868910081,1480550321000,2016-12-01,,,0,"[#notmypresident, #auditthevote, #grabyourwall...",804112446868910081,1,0,...,1,[],00:58:41,CET,#notmypresident #AuditTheVote #GrabYourWallet ...,[https://twitter.com/funder/status/80287292326...,798296697952735233,bonecho5,,


In [9]:
def get_edges():
    retweets = pd_data[pd_data.loc[:,'is_quote_status'].map(lambda q: q==1)]
    first_users = retweets.loc[:,'quote_url'].map(lambda quote:extract_username_from_link(quote))
    retweeters = retweets.loc[:,'username']
    edges = list(zip(first_users,retweeters))
    return edges

In [10]:
edges = get_edges()

In [11]:
# Will change this for loop latter
# Assign an id to the vertices (needed for the graph)
user_id = 0
map_user_id = {}
for edge in edges:
    for i in range(2):
        if edge[i] not in map_user_id.keys():
            map_user_id[edge[i]] = user_id
            user_id +=1

In [12]:
def edge_to_id(e):
    return (map_user_id[e[0]],map_user_id[e[1]])

In [13]:
# Builds the graph for the retweets
g = Graph()
g.add_vertices(user_id)
edges_id = list(map(lambda e: edge_to_id(e),edges))
g.add_edges(edges_id)

In [14]:
pr = g.pagerank()

In [15]:
ind = pr.index(np.max(pr))
most_influential = ''

for k,v in map_user_id.items():
    if v == ind:
        most_influential = k
        
print(most_influential)

realDonaldTrump


In [16]:
layout = g.layout("rt", 2)
plot(g, layout = layout).show()

In [17]:
pd_data.columns

Index(['conversation_id', 'created_at', 'date', 'gif_thumb', 'gif_url',
       'has_parent_tweet', 'hashtags', 'id', 'is_quote_status', 'is_reply_to',
       'likes_count', 'link', 'location', 'mentions', 'name', 'photos',
       'place', 'quote_id', 'quote_url', 'replies', 'replies_count', 'retweet',
       'retweets_count', 'tags', 'time', 'timezone', 'tweet', 'urls',
       'user_id', 'username', 'video_thumb', 'video_url'],
      dtype='object')

In [18]:
def get_all_tokens(tweet_list):
    """
    Generates tokens from tweets
    """
    all_text = ' '.join((t for t in tweet_list))
    tokens = (TweetTokenizer(preserve_case=False,
                            reduce_len=True,
                            strip_handles=False)
              .tokenize(all_text))
    # remove symbol-only tokens for now
    tokens = [tok for tok in tokens if not tok in string.punctuation]
    return tokens

In [19]:
tokens = get_all_tokens(pd_data.loc[:,'tweet'])

In [20]:
print('There are {0} tokens in total'.format(len(tokens)))

There are 1144055 tokens in total


In [21]:
top_grams = Counter(everygrams(tokens, min_len=2, max_len=4)).most_common(30)
print(top_grams) # most popular tokens

[(('…', '#notmypresident'), 5861), (('#whytepanther', '#notmypresident'), 2783), (('#notmypresident', '#auditthevote'), 2584), (('#notmypresident', '#notmypresident'), 2121), (('this', 'is'), 2119), (('of', 'the'), 1817), (('#auditthevote', '#notmypresident'), 1741), (('#notmypresident', '#theresistance'), 1678), (('is', 'a'), 1607), (('#notmypresident', '@realdonaldtrump'), 1469), (('donald', 'trump'), 1463), (('trump', 'is'), 1377), (('#notmypresident', '#resist'), 1377), (('in', 'the'), 1333), (('…', '@realdonaldtrump'), 1316), (('to', 'be'), 1252), (('@realdonaldtrump', '#notmypresident'), 1208), (('he', 'is'), 1192), (('is', '#notmypresident'), 1192), (('…', '#whytepanther'), 1151), (('…', '#whytepanther', '#notmypresident'), 1151), (('you', 'are'), 1146), (('#trump', '#notmypresident'), 1119), (('#theresistance', '#notmypresident'), 1050), (('#notmypresident', '#trump'), 1039), (('#notmypresident', '#nevertrump'), 1007), (('...', '#notmypresident'), 994), (('…', 'via'), 974), (('

In [22]:
languages = ['english',
             'spanish',
             'german',
             'french',
             'italian',
            ]
# Collect a list of stopwords in order to eleminate them from the 
stopw = list(set(it.chain.from_iterable((stopwords.words(lang)
                                                for lang in languages))))

In [23]:
# Generates punctuation words so that they can be removed from tokens along with stopwords

def gen_punc(max_length=4):

    def punc(length):
        return ((''.join(x) for x in it.product(string.punctuation,
                                                repeat=length)))
    words = it.chain.from_iterable((punc(length)
                                    for length in range(max_length+1)))
    return list(words)

In [24]:
# Final list of stopwords

stopw = list(it.chain(stopw, gen_punc(max_length=4)))
print(len(stopw))
print(stopw[:10])

1083488
['tenías', 'auraient', 'serons', 'unserem', 'vous', 'les', 'mais', 'ton', 'lo', 'avranno']


In [25]:
# Avoid tokenization of url by replacing them with <URL>
def replace_urls(in_string, replacement=None):
    replacement = '<-URL->' if replacement is None else replacement
    pattern = re.compile('(https?://)?(\w*[.]\w+)+([/?=&]+\w+)*')
    return re.sub(pattern, replacement, in_string)

In [29]:
def my_tokenizer(in_string):
    """
    Convert `in_string` of text to a list of tokens using NLTK's TweetTokenizer
    """
    tokenizer = TweetTokenizer(preserve_case=False,
                               reduce_len=True,
                               strip_handles=False)
    tokens = tokenizer.tokenize(in_string)
    return tokens

In [30]:
my_tokenizer(pd_data.loc[0,'tweet'])

['unfortunately',
 '@realdonaldtrump',
 "doesn't",
 'understand',
 'science',
 'or',
 'integrity',
 '.',
 '#notmypresident',
 '#dumptrump',
 'https://twitter.com/washingtonpost/status/804071010848931840',
 '…']

In [33]:
vec = TfidfVectorizer(preprocessor=replace_urls,
                      tokenizer=my_tokenizer,
                      stop_words=stopw,
                     )

In [34]:
vec

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function replace_urls at 0x7f01ac3f9b70>,
        smooth_idf=True,
        stop_words=['tenías', 'auraient', 'serons', 'unserem', 'vous', 'les', 'mais', 'ton', 'lo', 'avranno', 'fareste', 'dasselbe', 'mie', 'es', 'estuvieseis', 'une', 'hat', 'bist', 'yours', 'estaré', 'haven', 'algunos', 'jedem', 'fuera', 'eines', 'dell', 'estaríamos', 'sehr', 'sera', 'vuestra', 'coi', 'hu...>', '~~~?', '~~~@', '~~~[', '~~~\\', '~~~]', '~~~^', '~~~_', '~~~`', '~~~{', '~~~|', '~~~}', '~~~~'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function my_tokenizer at 0x7f01ac710378>, use_idf=True,
        vocabulary=None)